In [1]:
import psycopg2
from psycopg2.extensions import AsIs
try:
    conn =  psycopg2.connect("dbname=test_automated_segmentation user=postgres password = admin host=localhost")
except:
    print ("I am unable to connect to the database") 

In [2]:
'''
Given name of source table, union all rows in the source table,
create the new table and insert the union result into the 
first line of new table
'''
def create_new_map_table(conn,source_name, table_name):
    cur = conn.cursor()
    SQL_CREATE = '''
    DROP TABLE IF EXISTS %s;
    CREATE TABLE %s (
        gid SERIAL, 
        name VARCHAR(80),
        geom GEOMETRY,
        isLeaf BOOL,
        PRIMARY KEY (gid)
        
    );
    
    INSERT INTO %s (geom, isLeaf) VALUES (
        (SELECT ST_UNION(geom) AS geom FROM %s),
        TRUE);
    '''
    cur.execute(SQL_CREATE,(AsIs(table_name),AsIs(table_name),AsIs(table_name),AsIs(source_name)))
    conn.commit()

In [3]:
# create_new_map_table(conn,'la','a')
# create_new_map_table(conn,'ca','b')

In [4]:
def intersection(conn, existing_table, new_table):
    cur = conn.cursor()
    SQL_INTERSEC = '''
        INSERT INTO %s (geom) 
        SELECT ST_INTERSECTION(
            exist.geom, 
            ST_INTERSECTION(
                st_buffer(exist.geom, 0.0015), 
                st_buffer(new.geom, 0.0015)
            )
        )
        FROM %s exist, %s new
        WHERE exist.isLeaf = TRUE AND new.gid = 1;
    
    
    '''
    cur.execute(SQL_INTERSEC,(AsIs(existing_table),AsIs(existing_table),AsIs(new_table)))
    conn.commit()    

In [5]:
def difference(conn, existing_table, new_table):
    cur = conn.cursor()
    SQL_INTERSEC = '''
        INSERT INTO %s (geom)  
        SELECT ST_INTERSECTION(
            exist.geom, 
            ST_DIFFERENCE(
                st_buffer(exist.geom, 0.0015), 
                st_buffer(new.geom, 0.0015)
            )
        )
        FROM %s exist, %s new
        WHERE exist.isLeaf = TRUE AND new.gid = 1;
    
    
    '''
    cur.execute(SQL_INTERSEC,(AsIs(existing_table),AsIs(existing_table),AsIs(new_table)))
    conn.commit()    

In [6]:
def getNumRows(conn,table_name):
    cur = conn.cursor()
    SQL_NUMROW = '''
    SELECT COUNT(gid) 
    FROM %s
    '''
    cur.execute(SQL_NUMROW,(AsIs(table_name),))
    return cur.fetchone()[0]

In [7]:
def updateLeaves(conn, table_name, start, end):
    cur = conn.cursor()
    SQL_UPDATE_LEAVE = '''
    UPDATE %s 
    SET isLeaf = FALSE
    WHERE gid >= %s AND gid <= %s;
    UPDATE %s 
    SET isLeaf = TRUE
    WHERE gid > %s;
    '''
    cur.execute(SQL_UPDATE_LEAVE,(AsIs(table_name),start,end,AsIs(table_name),end))
    conn.commit()     

In [8]:
def segment_existing(conn,existing_table,new_table):
    curr_row = getNumRows(conn,existing_table)
    intersection(conn, existing_table, new_table)
    difference(conn, existing_table, new_table)
    updateLeaves(conn, existing_table, 1, curr_row)

In [9]:
create_new_map_table(conn,'la', 'a')
create_new_map_table(conn,'ca', 'b')
segment_existing(conn,'a','b')
create_new_map_table(conn,'usa', 'c')
segment_existing(conn,'a','c')